In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

## Load Data

In [2]:
vn_data = pd.read_csv('VN_converted.csv').drop(columns = ['Unnamed: 0'])
vn_data.shape

(5836, 9)

In [3]:
msk_data = pd.read_csv('MSK_converted.csv').drop(columns = ['Unnamed: 0'])
msk_data.shape

(9705, 9)

In [4]:
vn_data.columns == msk_data.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

## Train CROSS

In [5]:
data = msk_data
cross_data = vn_data

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

In [7]:
X, y_coord = data.iloc[:,:-3], data.iloc[:,-3:]

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_coord, y_test_coord = train_test_split(X, y_coord, test_size=0.2, random_state=123)
y_train = y_train_coord.iloc[:, 2]
y_test = y_test_coord.iloc[:, 2]

In [9]:
X_cross = cross_data.iloc[:,:-3]
y_cross = cross_data.iloc[:,-1]

In [10]:
# fit model no training data
models = {}
print('NB')
models['NB'] = GaussianNB()
models['NB'].fit(X_train, y_train)
print('xgb_train2')
models['xgb2'] = XGBClassifier(max_depth=2)
models['xgb2'].fit(X_train, y_train)
print('xgb_train3')
models['xgb3'] = XGBClassifier(max_depth=3)
models['xgb3'].fit(X_train, y_train)
print('xgb_train6')
models['xgb6'] = XGBClassifier(max_depth=6)
models['xgb6'].fit(X_train, y_train)
#print('svm')
#models['lin_svm'] = SVC(kernel="linear") #, C=0.025)
#models['lin_svm'].fit(X_train, y_train)

NB
xgb_train2
xgb_train3
xgb_train6


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
y_test_pred = {}
y_train_pred = {}
y_cross_pred = {}
test_predicted_coord = [y_test_coord]
for model_name, model in models.items():
    y_test_pred[model_name] = model.predict(X_test)
    y_train_pred[model_name] = model.predict(X_train)
    y_cross_pred[model_name] = model.predict(X_cross)
    test_predicted_coord.append(pd.DataFrame(model.predict_proba(X_test)[:,1], columns=[model_name]))
    print(model_name)
    accuracy = accuracy_score(y_test, y_test_pred[model_name])
    print("- accuracy test: %.2f%%" % (accuracy * 100.0))
    accuracy = accuracy_score(y_cross, y_cross_pred[model_name])
    print("- accuracy cross: %.2f%%" % (accuracy * 100.0))    
    accuracy = accuracy_score(y_train, y_train_pred[model_name])
    print("- accuracy train: %.2f%%" % (accuracy * 100.0))    

NB
- accuracy test: 72.75%
- accuracy cross: 79.11%
- accuracy train: 73.30%
xgb2
- accuracy test: 86.14%
- accuracy cross: 74.45%
- accuracy train: 86.15%
xgb3
- accuracy test: 86.45%
- accuracy cross: 72.17%
- accuracy train: 87.80%
xgb6
- accuracy test: 85.06%
- accuracy cross: 73.01%
- accuracy train: 95.09%


VN-> Msk test
<br>NB
- accuracy test: 79.02%
- accuracy cross: 61.21%
- accuracy train: 78.06%
<br>xgb2
- accuracy test: 88.18%
- accuracy cross: 60.43%
- accuracy train: 89.42%
<br>xgb3
- accuracy test: 89.55%
- accuracy cross: 59.53%
- accuracy train: 92.42%
<br>xgb6
- accuracy test: 92.04%
- accuracy cross: 56.94%
- accuracy train: 99.27%

Msk -> VN
<br>NB
- accuracy test: 72.75%
- accuracy cross: 79.11%
- accuracy train: 73.30%
<br>xgb2
- accuracy test: 86.14%
- accuracy cross: 74.45%
- accuracy train: 86.15%
<br>xgb3
- accuracy test: 86.45%
- accuracy cross: 72.17%
- accuracy train: 87.80%
<br>xgb6
- accuracy test: 85.06%
- accuracy cross: 73.01%
- accuracy train: 95.09%

In [12]:
all_cols = []
for i in range(len(test_predicted_coord)):
    print(test_predicted_coord[i].shape)
    all_cols+= list(test_predicted_coord[i].columns)

(1941, 3)
(1941, 1)
(1941, 1)
(1941, 1)
(1941, 1)


In [13]:
result = pd.DataFrame(np.concatenate(test_predicted_coord, axis=1), columns = all_cols)

In [14]:
result.to_csv('predict_result.csv')

In [15]:
#Xgb feature importance
importance = []
for i in range(len(X.columns)):
    importance.append((X.columns[i], models['xgb3'].feature_importances_[i]))
importance.sort(key = lambda x: x[1])
for item in importance:
    print(item[0], ': ', item[1])

Bump_HubDist :  0.017619444
Stop_HubDist :  0.031177206
traf_HubDist :  0.03318028
Cros_HubDist :  0.13449726
join_f_lanes :  0.2133873
join_speedlim :  0.5701385
